In [11]:
import pandas as pd
import re

In [41]:
df = pd.read_csv("dataset/sqad_split.csv", index_col=0)
df = df.drop(5146)
df["sentences"] = df.text.map(lambda x: x.split("\n"))
df.head()

,text,question,answer,answer_sentence,sentences
1,Létající jaguár je novela spisovatele Josefa F...,Kdo je autorem novely Létající jaguár?,Josefa Formánka,Létající jaguár je novela spisovatele Josefa F...,[Létající jaguár je novela spisovatele Josefa ...
2,"Houby (Fungi, dříve Mycophyta) představují vel...",Jak se nazývá věda zabývající se houbami?,mykologie,Věda zabývající se houbami se nazývá mykologie.,"[Houby (Fungi, dříve Mycophyta) představují ve..."
3,Čokoláda je obvyklá součást nejrůznějších druh...,"Jak se nazývá strom, jehož zrna jsou využívána...",Theobroma cacao,"Čokoláda se vyrábí z kvašených, pražených a ml...",[Čokoláda je obvyklá součást nejrůznějších dru...
4,Václav Havel (5. října 1936 Praha – 18. prosin...,Kdo se stal prvním prezidentem České republiky?,Václav Havel,Václav Havel (5. října 1936 Praha – 18. prosin...,[Václav Havel (5. října 1936 Praha – 18. prosi...
5,"Pampeliška (Taraxacum), či také smetánka, je z...",Do jaké čeledi rostlin patří pampeliška?,hvězdnicovité,"Pampeliška (Taraxacum), či také smetánka, je z...","[Pampeliška (Taraxacum), či také smetánka, je ..."


In [42]:
df.text.map(len).mean()


11994.540305819477

In [43]:
def get_sentence_index(row):
    try:
        return row.sentences.index(row.answer_sentence)
    except ValueError:
        return -1

#[df.answer_sentence.map(lambda x: "\n" not in x)]

In [44]:
for row in df[df.apply(get_sentence_index, axis=1) == -1].iloc:
    try:
        anss = row.answer_sentence.split("\n")
        sntcs = row.sentences

        i_beg = row.sentences.index(anss[0])
        i_end = row.sentences.index(anss[-1])

        sentences = sntcs[:i_beg] + [" ".join(sntcs[i_beg:i_end+1])] + sntcs[i_end+1:]
        answer = " ".join(anss)

        df.loc[row.name].sentences = sentences
        df.loc[row.name].answer_sentence = answer
        print(f"{row.name: 6d}", len(answer.split(" ")))
    except ValueError:
        print(repr(row.sentences))
        print(repr(row.answer_sentence))
        print()


     8 16
   433 16
   466 21
   600 11
   630 41
   636 29
   652 18
   825 19
   931 79
   953 61
   954 61
   955 61
  1307 29
  1972 20
  2439 20
  2441 20
  2702 73
  5260 35
  6623 144
  8264 22
  9356 24
  9927 62
 10026 23
 10768 21
 11516 28
 11525 25
 11576 54
 11719 38
 13341 31
 13346 79
 13368 50
 13444 31


In [45]:
starts = df.apply(get_sentence_index, axis=1)
df["start"] = starts
df[starts == -1]

,text,question,answer,answer_sentence,sentences,start


In [46]:
starts.value_counts()

0      6445
1      1214
2       757
3       492
4       364
       ... 
521       1
239       1
197       1
109       1
511       1
Length: 315, dtype: int64

In [88]:
from random import randint, random

def create_training_data(df, margin=5, dupe=1.0):
    index = 0
    data = []
    out = pd.DataFrame(columns=["text", "question", "answer", "answer_sentence", "pos"])
    for row in df.iloc:
        curdupe = dupe
        pars = []
        
        while random() <= curdupe:
            pre = randint(0, margin)
            post = randint(margin-pre, 2*margin-pre)
            pos = row.start
            
            l = pos-pre
            r = pos+post+1
            slen = len(row.sentences)
            
            if r - l > slen:
                text_sntcs = row.sentences
#                 print(slen, text_sntcs)
            else:
                text_sntcs = []
                if l < 0:
                    text_sntcs += row.sentences[l:]
                    text_sntcs += row.sentences[:pos]
                else:
                    text_sntcs += row.sentences[l:pos]

                if r > slen:
                    text_sntcs += row.sentences[:r-slen]
                text_sntcs += row.sentences[pos:r]

                if not r-l == len(text_sntcs):
                    print(l, pos, r, slen, r-l, len(text_sntcs))

#             print(len(" ".join(text_sntcs).split(" ")))


            answer = row.answer_sentence
            context = " ".join(text_sntcs)
            answer_start = re.search(re.escape(answer), context).start()
            
            par = {
                "qas": [{
                    "id": f"{row.name}.{len(pars)}",
                    "question": row.question,
                    "answers": [{
                        "text": answer,
                        "answer_start": answer_start
                    }],
                    "is_impossible": False
                }],
                "context": context
            }
            
            if r - l > slen:
                break
        
            pars.append(par)
            curdupe -= 1
        
        data.append({"title": str(row.name), "paragraphs": pars})
        
    return data

data = create_training_data(df, 5, 2)

In [89]:
import json
def write_squad_json(path, data, version="42.0"):
    jj = {
        "version": version,
        "data": data
    }
    
    with open(path, "w") as f:
        json.dump(jj, f)
        
write_squad_json("dataset/sqad.json", data)